In [3]:
!pip install thop ptflops

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from thop import profile
from collections import Counter
from glob import glob
import glob as glob_module
import time
import os
import re
import gc
from IPython.display import display
from torch.amp import autocast, GradScaler

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

if torch.cuda.is_available():
    torch.cuda.empty_cache()


def clean_state_dict(state_dict):
    return {k: v for k, v in state_dict.items()
            if 'total_ops' not in k and 'total_params' not in k}


class UCIHARDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


def read_txt_matrix(file_path):
    return np.loadtxt(file_path)


def load_uci_har(root_path='/content/drive/MyDrive/HAR_Dataset/UCI'):
    print(f"\n[UCI-HAR] Loading from: {root_path}")
    UCI_CHANNELS_PREFIX = [
        "total_acc_x_", "total_acc_y_", "total_acc_z_",
        "body_acc_x_", "body_acc_y_", "body_acc_z_",
        "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
    ]
    def load_split(split):
        channels = []
        for idx, prefix in enumerate(UCI_CHANNELS_PREFIX):
            file_path = os.path.join(root_path, f"{prefix}{split}.txt")
            print(f"  [{idx+1}/{len(UCI_CHANNELS_PREFIX)}] Loading: {os.path.basename(file_path)}")
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")
            channels.append(read_txt_matrix(file_path))
        X = np.stack(channels, axis=1)
        y = read_txt_matrix(os.path.join(root_path, f"y_{split}.txt")).astype(int) - 1
        return X, y
    X_train, y_train = load_split('train')
    X_test, y_test = load_split('test')
    X_train = X_train.transpose(0, 2, 1)
    X_test = X_test.transpose(0, 2, 1)
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    scaler = StandardScaler()
    scaler.fit(X_train_flat)
    X_train_flat = scaler.transform(X_train_flat)
    X_test_flat = scaler.transform(X_test_flat)
    X_train = X_train_flat.reshape(X_train.shape)
    X_test = X_test_flat.reshape(X_test.shape)
    activity_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train.astype(np.int64), X_test, y_test.astype(np.int64), activity_names


def load_wisdm_data(dataset_path="/content/drive/MyDrive/HAR_Dataset/WISDM"):
    print(f"\n[WISDM] Loading from: {dataset_path}")
    if os.path.isfile(dataset_path):
        file_paths = [dataset_path]
    else:
        possible_files = ['WISDM_ar_v1.1_raw.txt', 'WISDM_ar_v1.1_trans.arff', 'wisdm-dataset.txt', 'actitracker_raw.txt']
        file_paths = []
        for filename in possible_files:
            full_path = os.path.join(dataset_path, filename)
            if os.path.exists(full_path):
                file_paths.append(full_path)
                print(f"  Found: {filename}")
    if not file_paths:
        return None, None, None, None, None
    all_data = []
    for idx, file_path in enumerate(file_paths):
        print(f"  [{idx+1}/{len(file_paths)}] Processing: {os.path.basename(file_path)}")
        with open(file_path, 'r') as f:
            lines = f.readlines()
        cleaned_data = []
        for line in lines:
            line = line.strip()
            if not line:
                continue
            line = line.rstrip(';').rstrip(',')
            if ',' in line:
                parts = line.split(',')
            elif ';' in line:
                parts = line.split(';')
            else:
                continue
            if len(parts) < 6:
                continue
            try:
                user = parts[0].strip()
                activity = parts[1].strip()
                timestamp = parts[2].strip()
                x_str = parts[3].strip()
                y_str = parts[4].strip()
                z_str = parts[5].strip()
                if ';' in x_str:
                    x_str = x_str.split(';')[0]
                if ';' in y_str:
                    y_str = y_str.split(';')[0]
                if ';' in z_str:
                    z_str = z_str.split(';')[0]
                x = float(x_str)
                y = float(y_str)
                z = float(z_str)
                cleaned_data.append([user, activity, timestamp, x, y, z])
            except (ValueError, IndexError):
                continue
        if cleaned_data:
            df = pd.DataFrame(cleaned_data, columns=['user', 'activity', 'timestamp', 'x', 'y', 'z'])
            df['x'] = pd.to_numeric(df['x'], errors='coerce')
            df['y'] = pd.to_numeric(df['y'], errors='coerce')
            df['z'] = pd.to_numeric(df['z'], errors='coerce')
            df = df.dropna()
            all_data.append(df)
    if not all_data:
        return None, None, None, None, None
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.dropna()
    combined_df = combined_df[combined_df['activity'].str.strip() != '']
    all_windows = []
    all_labels = []
    groups = combined_df.groupby(['user', 'activity']) if 'user' in combined_df.columns else combined_df.groupby(['activity'])
    window_size = 80
    step = 40
    for group_name, group_data in groups:
        activity = group_name[-1] if isinstance(group_name, tuple) else group_name
        acc_data = group_data[['x', 'y', 'z']].values.astype(np.float32)
        if len(acc_data) < window_size:
            continue
        start = 0
        while start + window_size <= len(acc_data):
            window_data = acc_data[start:start + window_size, :]
            all_windows.append(window_data)
            all_labels.append(activity)
            start += step
    if not all_windows:
        return None, None, None, None, None
    X_windowed = np.array(all_windows, dtype=np.float32)
    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(all_labels)
    class_names = [str(label) for label in label_encoder.classes_]
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, class_names


def load_dsads_data(data_path, w_s=25, stride=12):
    print(f"\n[DSADS] Loading from: {data_path}")
    all_windows = []
    all_labels = []
    activities = {
        'a01': 'sitting', 'a02': 'standing', 'a03': 'lying on back',
        'a04': 'lying on right side', 'a05': 'ascending stairs',
        'a06': 'descending stairs', 'a07': 'standing in an elevator still',
        'a08': 'moving around in an elevator', 'a09': 'walking in a parking lot',
        'a10': 'walking on a treadmill with a speed of 4 kmh',
        'a11': 'walking in flat and 15 deg inclined positions',
        'a12': 'running on a treadmill with a speed of 8 kmh',
        'a13': 'exercising on a stepper', 'a14': 'exercising on a cross trainer',
        'a15': 'cycling on an exercise bike in horizontal positions',
        'a16': 'cycling on an exercise bike in vertical positions',
        'a17': 'rowing', 'a18': 'jumping', 'a19': 'playing basketball'
    }
    activity_codes = sorted(activities.keys())
    label_encoder = LabelEncoder()
    label_encoder.fit(activity_codes)
    persons = ['p' + str(i) for i in range(1, 9)]
    total_activities = len(activity_codes) * len(persons)
    progress = 0
    for person_str in persons:
        for activity_str in activity_codes:
            progress += 1
            print(f"  [{progress}/{total_activities}] Processing: {person_str} - {activity_str}")
            activity_label = label_encoder.transform([activity_str])[0]
            pattern = os.path.join(data_path, activity_str, person_str, 's*.txt')
            segment_files = sorted(glob(pattern))
            if not segment_files:
                continue
            for f in segment_files[:11]:
                try:
                    segment_data = np.loadtxt(f, delimiter=',')
                    if segment_data.shape[0] < w_s or segment_data.shape[1] < 45:
                        continue
                    segment_data = np.nan_to_num(segment_data, nan=0.0)
                    start = 0
                    while start + w_s <= segment_data.shape[0]:
                        window_data = segment_data[start : start + w_s, :]
                        all_windows.append(window_data)
                        all_labels.append(activity_label)
                        start += stride
                except:
                    continue
    if not all_windows:
        return None, None, None, None, None
    X_windowed = np.array(all_windows, dtype=np.float32)
    y_encoded = np.array(all_labels, dtype=int)
    scaler = StandardScaler()
    X_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_flat = scaler.fit_transform(X_flat)
    X_windowed = X_flat.reshape(X_windowed.shape)
    activity_names_sorted = [activities[code] for code in label_encoder.classes_]
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_names_sorted


def load_pamap2_data(dataset_dir="/content/drive/MyDrive/HAR_Dataset/PAMAP2"):
    print(f"\n[PAMAP2] Loading from: {dataset_dir}")
    file_paths = sorted(glob(os.path.join(dataset_dir, 'Protocol', 'subject*.dat')))
    optional_path = os.path.join(dataset_dir, 'Optional')
    if os.path.exists(optional_path):
        file_paths += sorted(glob(os.path.join(optional_path, 'subject*.dat')))
    if not file_paths:
        return None, None, None, None, None
    print(f"  Found {len(file_paths)} subject files")
    activity_labels = [
        "lying", "sitting", "standing", "walking", "running", "cycling",
        "Nordic walking", "ascending stairs", "descending stairs",
        "vacuum cleaning", "ironing", "rope jumping"
    ]
    label_to_activity_idx = {
        1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 12: 7, 13: 8, 16: 9, 17: 10, 24: 11
    }
    all_windows = []
    all_labels = []
    window_size = 100
    step = 50
    for idx, file_path in enumerate(file_paths):
        print(f"  [{idx+1}/{len(file_paths)}] Processing: {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path, sep=r'\s+', header=None, na_values='NaN')
        except:
            continue
        df_cleaned = df.ffill().bfill()
        if df_cleaned.empty:
            continue
        labels = df_cleaned.iloc[:, 1].values.astype(int)
        all_sensor_cols = list(range(4, 10)) + list(range(21, 27)) + list(range(38, 44))
        if df_cleaned.shape[1] < max(all_sensor_cols) + 1:
            continue
        features = df_cleaned.iloc[:, all_sensor_cols].values.astype(np.float32)
        valid_indices = np.where(np.isin(labels, list(label_to_activity_idx.keys())))[0]
        if len(valid_indices) == 0:
            continue
        features = features[valid_indices, :]
        labels = labels[valid_indices]
        if len(features) < window_size:
            continue
        start = 0
        while start + window_size <= len(features):
            window_data = features[start : start + window_size, :]
            window_labels_raw = labels[start : start + window_size]
            most_common_label = Counter(window_labels_raw).most_common(1)[0][0]
            if most_common_label in label_to_activity_idx:
                all_windows.append(window_data)
                all_labels.append(label_to_activity_idx[most_common_label])
            start += step
    if not all_windows:
        return None, None, None, None, None
    X_windowed = np.array(all_windows, dtype=np.float32)
    y_encoded = np.array(all_labels, dtype=int)
    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_labels


def load_mhealth_data(dataset_dir="/content/drive/MyDrive/HAR_Dataset/MHEALTH"):
    print(f"\n[MHEALTH] Loading from: {dataset_dir}")
    if not os.path.exists(dataset_dir):
        return None, None, None, None, None
    subject_files = sorted([
        os.path.join(dataset_dir, f)
        for f in os.listdir(dataset_dir)
        if f.startswith("mHealth_subject") and f.endswith(".log")
    ])
    if not subject_files:
        return None, None, None, None, None
    print(f"  Found {len(subject_files)} subject files")
    all_windows = []
    all_labels = []
    window_size = 50
    step = 25
    for idx, file_path in enumerate(subject_files):
        print(f"  [{idx+1}/{len(subject_files)}] Processing: {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path, sep=r'\s+', header=None, engine='python', dtype=np.float32)
            df = df.ffill().bfill()
            if df.shape[1] < 24:
                continue
            labels = df.iloc[:, 23].values.astype(int)
            imu_cols = [0, 1, 2] + list(range(5, 23))
            features = df.iloc[:, imu_cols].values
            valid_indices = np.where(labels != 0)[0]
            if len(valid_indices) == 0:
                continue
            features = features[valid_indices, :]
            labels = labels[valid_indices]
            if len(features) < window_size:
                continue
            start = 0
            while start + window_size <= len(features):
                window_data = features[start : start + window_size, :]
                window_labels_raw = labels[start : start + window_size]
                most_common_label = Counter(window_labels_raw).most_common(1)[0][0]
                all_windows.append(window_data)
                all_labels.append(most_common_label)
                start += step
        except:
            continue
    if not all_windows:
        return None, None, None, None, None
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(all_labels)
    X_windowed = np.array(all_windows, dtype=np.float32)
    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    mhealth_activity_mapping = {
        1: 'Standing still', 2: 'Sitting and relaxing', 3: 'Lying down', 4: 'Walking',
        5: 'Climbing stairs', 6: 'Waist bends forward', 7: 'Frontal elevation of arms',
        8: 'Knees bending', 9: 'Cycling', 10: 'Jogging', 11: 'Running', 12: 'Jump front & back'
    }
    activity_labels = []
    class_names = list(label_encoder.classes_)
    for encoded_idx in range(len(class_names)):
        original_label = class_names[encoded_idx]
        activity_labels.append(mhealth_activity_mapping.get(original_label, f"Unknown_Activity_{original_label}"))
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_labels


def slide_window(data, w_s, stride):
    windows = []
    start = 0
    while start + w_s <= len(data):
        windows.append(data[start:start + w_s, :])
        start += stride
    if not windows:
        return np.array([])
    return np.array(windows)


def load_mobiact_sensor_file(path):
    data = []
    try:
        with open(path, 'r') as f:
            in_data_section = False
            for line in f:
                if line.startswith('@DATA'):
                    in_data_section = True
                    continue
                if not in_data_section:
                    continue
                parts = line.strip().split(',')
                if len(parts) == 4:
                    try:
                        data.append([float(p) for p in parts])
                    except ValueError:
                        continue
    except Exception as e:
        return None
    if not data:
        return None
    return np.array(data, dtype=np.float32)


def load_mobiact_data(data_path, w_s=100, stride=50):
    print(f"\n[MobiAct] Loading from: {data_path}")
    ADL_CODES = ['STD', 'WAL', 'JOG', 'JUM', 'STU', 'STN', 'SCH', 'CSI', 'CSO']
    ALL_ACTIVITIES = ADL_CODES
    label_encoder = LabelEncoder()
    label_encoder.fit(ALL_ACTIVITIES)
    activity_names = list(label_encoder.classes_)
    subject_dirs = sorted(glob_module.glob(os.path.join(data_path, 'sub*')))
    subjects_to_use = []
    for sub_dir in subject_dirs:
        if os.path.isdir(os.path.join(sub_dir, 'ADL')):
            subjects_to_use.append(os.path.basename(sub_dir))
    print(f"  Found {len(subjects_to_use)} subjects")
    all_data_for_scaler = []
    all_windows, all_labels = [], []
    temp_data_store = []
    for sub_idx, sub_name in enumerate(subjects_to_use):
        print(f"  [{sub_idx+1}/{len(subjects_to_use)}] Processing: {sub_name}")
        subject_id_match = re.search(r'(\d+)', sub_name)
        if not subject_id_match:
            continue
        subject_id = int(subject_id_match.group(0))
        for act_type_folder in ['ADL']:
            act_folders_path = os.path.join(data_path, sub_name, act_type_folder, '*')
            for act_folder_path in glob_module.glob(act_folders_path):
                act_code = os.path.basename(act_folder_path)
                if act_code not in ALL_ACTIVITIES:
                    continue
                label = label_encoder.transform([act_code])[0]
                trial_files = glob_module.glob(os.path.join(act_folder_path, f'{act_code}_acc_{subject_id}_*.txt'))
                trials = sorted(list(set([f.split('_')[-1].split('.')[0] for f in trial_files])))
                for trial in trials:
                    acc_file = os.path.join(act_folder_path, f'{act_code}_acc_{subject_id}_{trial}.txt')
                    gyro_file = os.path.join(act_folder_path, f'{act_code}_gyro_{subject_id}_{trial}.txt')
                    ori_file = os.path.join(act_folder_path, f'{act_code}_ori_{subject_id}_{trial}.txt')
                    if not all(os.path.exists(f) for f in [acc_file, gyro_file, ori_file]):
                        continue
                    data_acc = load_mobiact_sensor_file(acc_file)
                    data_gyro = load_mobiact_sensor_file(gyro_file)
                    data_ori = load_mobiact_sensor_file(ori_file)
                    if any(d is None for d in [data_acc, data_gyro, data_ori]):
                        continue
                    L = min(len(data_acc), len(data_gyro), len(data_ori))
                    if L < w_s:
                        continue
                    combined_data = np.hstack((data_acc[:L, 1:], data_gyro[:L, 1:], data_ori[:L, 1:]))
                    combined_data = combined_data.astype(np.float32)
                    combined_data = pd.DataFrame(combined_data).ffill().bfill().values
                    if np.isnan(combined_data).any():
                        continue
                    all_data_for_scaler.append(combined_data)
                    temp_data_store.append({'data': combined_data, 'label': label})
    if not all_data_for_scaler:
        return None, None, None, None, None
    scaler = StandardScaler().fit(np.vstack(all_data_for_scaler))
    for item in temp_data_store:
        scaled_data = scaler.transform(item['data'])
        scaled_data = np.nan_to_num(scaled_data)
        windows = slide_window(scaled_data, w_s, stride)
        if windows.shape[0] > 0:
            all_windows.append(windows)
            all_labels.extend([item['label']] * windows.shape[0])
    if not all_windows:
        return None, None, None, None, None
    X_windowed = np.vstack(all_windows).astype(np.float32)
    X_windowed = np.transpose(X_windowed, (0, 2, 1))
    y_encoded = np.array(all_labels, dtype=int)
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_names


class MotionSenseLoader:
    def __init__(self, frame_len, feature_name, N_classes):
        self.feature_names = feature_name
        self.N_Feature = len(feature_name)
        self.frame_length = frame_len
        self.hop_size = frame_len//2
        self.N_classes = N_classes
        self.label_encoder = OneHotEncoder(sparse_output=False)
    def framing(self, signal):
        shape = ((signal.shape[0] - self.frame_length) // self.hop_size + 1, self.frame_length)
        strides = (signal.strides[0] * self.hop_size, signal.strides[0])
        return np.lib.stride_tricks.as_strided(signal, shape=shape, strides=strides)
    def create_label(self, label): return self.label_encoder.fit_transform(label)
    def load_trainings_data(self, files, label_frame):
        label = self.create_label(label_frame)
        self.trainings_data, self.trainings_label = self.load_data(files, label)
    def load_validation_data(self, files, label_frame):
        label = self.label_encoder.transform(label_frame)
        self.validation_data, self.validation_label = self.load_data(files, label)
    def load_data(self, files, label):
        feature_matrix, label_matrix = None, None
        for i in range(len(files)):
            if i % 50 == 0:
                print(f"    Processing file {i+1}/{len(files)}")
            try: tmp_data = pd.read_csv(files[i], engine='python')
            except: continue
            N_Blocks = 1+(np.shape(tmp_data)[0]-self.frame_length)//self.hop_size
            if N_Blocks <= 0: continue
            tmp_feature_mat = np.zeros((N_Blocks, self.frame_length, self.N_Feature))
            tmp_label_vec = np.zeros((N_Blocks, self.N_classes))
            for j in range(N_Blocks): tmp_label_vec[j, :] = label[i, :]
            for idf, feat in enumerate(self.feature_names):
                frame_matrix = self.framing(tmp_data[feat].to_numpy())
                tmp_feature_mat[:, :, idf] = frame_matrix[:N_Blocks]
            if feature_matrix is None:
                feature_matrix = tmp_feature_mat
                label_matrix = tmp_label_vec
            else:
                feature_matrix = np.append(feature_matrix, tmp_feature_mat, axis=0)
                label_matrix = np.append(label_matrix, tmp_label_vec, axis=0)
        return feature_matrix, label_matrix


def load_motionsense_data(root_path='/content/drive/MyDrive/HAR_Dataset/MOTIONSENSE'):
    print(f"\n[MotionSense] Loading from: {root_path}")
    files, label = [], []
    for dirname, _, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.endswith('.csv') and not filename.startswith('.'):
                full_path = os.path.join(dirname, filename)
                if 'sub_' in filename:
                    files.append(full_path)
                    parent_dir = os.path.basename(os.path.dirname(full_path))
                    if '_' in parent_dir: label.append(parent_dir.split('_')[0])
                    else: files.pop()
    if not files: return None, None, None, None, None
    print(f"  Found {len(files)} CSV files")
    label_frame = pd.DataFrame(label, columns=['act'])
    files_train, files_valid, y_train_raw, y_valid_raw = train_test_split(files, label_frame, test_size=0.2, random_state=0)
    Feature = ['attitude.roll','attitude.pitch','attitude.yaw','gravity.x','gravity.y','gravity.z',
               'rotationRate.x','rotationRate.y','rotationRate.z','userAcceleration.x','userAcceleration.y','userAcceleration.z']
    N_classes = 6
    loader = MotionSenseLoader(128, Feature, N_classes)
    print(f"  Processing training files...")
    loader.load_trainings_data(files_train, y_train_raw)
    print(f"  Processing validation files...")
    loader.load_validation_data(files_valid, y_valid_raw)
    X_train = loader.trainings_data.astype(np.float32).transpose(0, 2, 1)
    X_test = loader.validation_data.astype(np.float32).transpose(0, 2, 1)
    y_train = np.argmax(loader.trainings_label, axis=1)
    y_test = np.argmax(loader.validation_label, axis=1)
    activity_names = list(loader.label_encoder.categories_[0])
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_names


def load_unimib_shar_data(dataset_dir="/content/drive/MyDrive/HAR_Dataset/UNIMIB (1)"):
    print(f"\n[UNIMIB] Loading from: {dataset_dir}")
    train_path = os.path.join(dataset_dir, "unimib_train.csv")
    test_path = os.path.join(dataset_dir, "unimib_test.csv")
    val_path = os.path.join(dataset_dir, "unimib_val.csv")
    if not os.path.exists(train_path): return None, None, None, None, None
    def process_unimib_csv(path):
        if not os.path.exists(path): return np.array([]), np.array([])
        print(f"  Processing: {os.path.basename(path)}")
        df = pd.read_csv(path)
        df = df.sort_values(by=['ID', 't'])
        X_list, y_list = [], []
        for _, group in df.groupby('ID'):
            X_list.append(group[['ax', 'ay', 'az']].values.astype(np.float32))
            y_list.append(group['label'].iloc[0])
        return np.array(X_list), np.array(y_list)
    X_train, y_train_raw = process_unimib_csv(train_path)
    X_test, y_test_raw = process_unimib_csv(test_path)
    if os.path.exists(val_path):
        X_val, y_val_raw = process_unimib_csv(val_path)
        if len(X_val) > 0:
            X_test = np.concatenate((X_test, X_val), axis=0)
            y_test_raw = np.concatenate((y_test_raw, y_val_raw), axis=0)
    le = LabelEncoder()
    y_train = le.fit_transform(y_train_raw)
    y_test = le.transform(y_test_raw)
    class_names = [str(c) for c in le.classes_]
    B_train, T, C = X_train.shape
    B_test, _, _ = X_test.shape
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(B_train, -1)).reshape(B_train, T, C).transpose(0, 2, 1)
    X_test = scaler.transform(X_test.reshape(B_test, -1)).reshape(B_test, T, C).transpose(0, 2, 1)
    print(f"  Completed: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test, y_test, class_names


SENSOR_CONFIG = {
    'UCI-HAR': {'acc_indices': [0, 1, 2, 3, 4, 5], 'gyro_indices': [6, 7, 8]},
    'WISDM': {'acc_indices': [0, 1, 2], 'gyro_indices': []},
    'DSADS': {'acc_indices': [0, 1, 2, 9, 10, 11, 18, 19, 20, 27, 28, 29, 36, 37, 38], 'gyro_indices': [3, 4, 5, 12, 13, 14, 21, 22, 23, 30, 31, 32, 39, 40, 41]},
    'PAMAP2': {'acc_indices': [0, 1, 2, 6, 7, 8, 12, 13, 14], 'gyro_indices': [3, 4, 5, 9, 10, 11, 15, 16, 17]},
    'MHEALTH': {'acc_indices': [0, 1, 2, 9, 10, 11], 'gyro_indices': [6, 7, 8, 15, 16, 17]},
    'MOBIACT': {'acc_indices': [0, 1, 2], 'gyro_indices': [3, 4, 5]},
    'MOTIONSENSE': {'acc_indices': [3, 4, 5, 9, 10, 11], 'gyro_indices': [6, 7, 8]},
    'UNIMIB': {'acc_indices': [0, 1, 2], 'gyro_indices': []},
}


class RelativeEnergyPhysics(nn.Module):
    def __init__(self, acc_indices, gyro_indices, learnable=True):
        super().__init__()
        self.acc_indices = acc_indices
        self.gyro_indices = gyro_indices
        self.m = nn.Parameter(torch.tensor(1.0), requires_grad=learnable)
        self.I = nn.Parameter(torch.tensor(1.0), requires_grad=learnable)

    def forward(self, x):
        m_pos = F.softplus(self.m)
        I_pos = F.softplus(self.I)
        if len(self.acc_indices) > 0:
            acc_data = x[:, :, self.acc_indices]
            n_acc_sensors = len(self.acc_indices) // 3
            if n_acc_sensors > 0:
                acc_reshaped = acc_data.view(x.shape[0], x.shape[1], n_acc_sensors, 3)
                acc_magnitude = (acc_reshaped ** 2).sum(dim=-1).mean(dim=-1, keepdim=True)
            else:
                acc_magnitude = torch.zeros(x.shape[0], x.shape[1], 1, device=x.device)
        else:
            acc_magnitude = torch.zeros(x.shape[0], x.shape[1], 1, device=x.device)
        E_kinetic_proxy = 0.5 * m_pos * acc_magnitude
        if len(self.gyro_indices) > 0:
            gyro_data = x[:, :, self.gyro_indices]
            n_gyro_sensors = len(self.gyro_indices) // 3
            if n_gyro_sensors > 0:
                gyro_reshaped = gyro_data.view(x.shape[0], x.shape[1], n_gyro_sensors, 3)
                gyro_magnitude = (gyro_reshaped ** 2).sum(dim=-1).mean(dim=-1, keepdim=True)
            else:
                gyro_magnitude = torch.zeros(x.shape[0], x.shape[1], 1, device=x.device)
        else:
            gyro_magnitude = torch.zeros(x.shape[0], x.shape[1], 1, device=x.device)
        E_rotational_proxy = 0.5 * I_pos * gyro_magnitude
        E_relative = E_kinetic_proxy + E_rotational_proxy
        return E_relative


class PotentialEnergyField(nn.Module):
    def __init__(self, input_dim, hidden_dim, sensor_config, use_physics=True, learnable_physics=True):
        super().__init__()
        self.use_physics = use_physics
        if use_physics:
            self.physics = RelativeEnergyPhysics(
                acc_indices=sensor_config['acc_indices'],
                gyro_indices=sensor_config['gyro_indices'],
                learnable=learnable_physics
            )
        self.energy_net = nn.Sequential(
            nn.Linear(input_dim + (1 if use_physics else 0), hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        if self.use_physics:
            E_physics = self.physics(x)
            x_augmented = torch.cat([x, E_physics], dim=-1)
        else:
            x_augmented = x
        E_learned = self.energy_net(x_augmented)
        return E_learned


class EnergyGradientFlow(nn.Module):
    def __init__(self, input_dim, hidden_dim, sensor_config, use_physics=True, use_physics_gradient=True, learnable_physics=True):
        super().__init__()
        self.acc_indices = sensor_config['acc_indices']
        self.gyro_indices = sensor_config['gyro_indices']
        self.input_dim = input_dim
        self.use_physics = use_physics
        self.use_physics_gradient = use_physics_gradient
        if use_physics:
            self.physics = RelativeEnergyPhysics(
                acc_indices=sensor_config['acc_indices'],
                gyro_indices=sensor_config['gyro_indices'],
                learnable=learnable_physics
            )
        self.gradient_net = nn.Sequential(
            nn.Linear(input_dim + (1 if use_physics else 0), hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        if self.use_physics:
            E_physics = self.physics(x)
            x_augmented = torch.cat([x, E_physics], dim=-1)
        else:
            x_augmented = x
        gradient = self.gradient_net(x_augmented)
        if self.use_physics and self.use_physics_gradient:
            m_pos = F.softplus(self.physics.m)
            I_pos = F.softplus(self.physics.I)
            physics_gradient = torch.zeros_like(x)
            if len(self.acc_indices) > 0:
                acc_data = x[:, :, self.acc_indices]
                force_proxy = acc_data * m_pos
                physics_gradient[:, :, self.acc_indices] = force_proxy
            if len(self.gyro_indices) > 0:
                gyro_data = x[:, :, self.gyro_indices]
                torque_proxy = gyro_data * I_pos
                physics_gradient[:, :, self.gyro_indices] = torque_proxy
            return gradient + 0.1 * physics_gradient
        return gradient


class LandscapeGeometryEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        combined_dim = input_dim * 2 + 1
        self.input_proj = nn.Conv1d(combined_dim, hidden_dim, kernel_size=1)
        self.conv = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, padding=2)
        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(0.2)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=8, batch_first=True)

    def forward(self, energy, gradient, x_original):
        landscape_state = torch.cat([x_original, gradient, energy], dim=-1)
        x = self.input_proj(landscape_state.transpose(1, 2))
        x = self.conv(x)
        x = self.norm(x.transpose(1, 2)).transpose(1, 2)
        x = F.gelu(x)
        x = self.dropout(x)
        h_attn_in = x.transpose(1, 2)
        h_attn_out, _ = self.attention(h_attn_in, h_attn_in, h_attn_in)
        return h_attn_out


class SimpleLandscapeEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        combined_dim = input_dim * 2 + 1
        self.encoder = nn.Sequential(
            nn.Conv1d(combined_dim, hidden_dim, kernel_size=5, padding=2),
            nn.LayerNorm([hidden_dim]),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, padding=2),
            nn.LayerNorm([hidden_dim]),
            nn.GELU()
        )

    def forward(self, energy, gradient, x_original):
        landscape_state = torch.cat([x_original, gradient, energy], dim=-1)
        x = self.encoder(landscape_state.transpose(1, 2))
        return x.transpose(1, 2)


class MIELHAR(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, sensor_config,
                 use_physics=True, learnable_physics=True, use_physics_gradient=True,
                 use_landscape=True, use_energy_loss=True):
        super().__init__()
        self.use_energy_loss = use_energy_loss
        self.potential_field = PotentialEnergyField(input_dim, hidden_dim, sensor_config, use_physics, learnable_physics)
        self.gradient_flow = EnergyGradientFlow(input_dim, hidden_dim, sensor_config, use_physics, use_physics_gradient, learnable_physics)
        if use_landscape:
            self.landscape_encoder = LandscapeGeometryEncoder(input_dim, hidden_dim)
        else:
            self.landscape_encoder = SimpleLandscapeEncoder(input_dim, hidden_dim)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )
        self.energy_reg_weight = 0.01

    def forward(self, x, return_energy=False):
        energy = self.potential_field(x)
        gradient = self.gradient_flow(x)
        landscape_features = self.landscape_encoder(energy, gradient, x)
        global_features = self.global_pool(landscape_features.transpose(1, 2)).squeeze(-1)
        logits = self.classifier(global_features)
        if return_energy:
            return logits, energy, gradient
        return logits

    def extract_features(self, x):
        energy = self.potential_field(x)
        gradient = self.gradient_flow(x)
        landscape_features = self.landscape_encoder(energy, gradient, x)
        global_features = self.global_pool(landscape_features.transpose(1, 2)).squeeze(-1)
        return global_features

    def compute_energy_loss(self, energy, gradient, labels):
        energy_diff = energy[:, 1:] - energy[:, :-1]
        smoothness_loss = torch.mean(energy_diff ** 2)
        gradient_mag = torch.norm(gradient, dim=-1)
        gradient_loss = torch.mean(gradient_mag)
        return smoothness_loss + 0.1 * gradient_loss


def plot_tsne(features, labels, activity_names, save_path, samples_per_class=600):
    sampled_features, sampled_labels = [], []
    for i in range(len(activity_names)):
        class_mask = (labels == i)
        class_indices = np.where(class_mask)[0]
        if len(class_indices) > 0:
            if len(class_indices) >= samples_per_class:
                selected_indices = np.random.choice(class_indices, samples_per_class, replace=False)
            else:
                selected_indices = class_indices
            sampled_features.append(features[selected_indices])
            sampled_labels.append(labels[selected_indices])
    features_sampled = np.vstack(sampled_features)
    labels_sampled = np.concatenate(sampled_labels)
    n_samples = features_sampled.shape[0]
    perplexity = min(30, n_samples - 1)
    features_2d = TSNE(n_components=2, perplexity=perplexity, learning_rate=200, random_state=42).fit_transform(features_sampled)
    num_classes = len(activity_names)
    cmap = plt.cm.get_cmap('tab20', num_classes)
    colors = [cmap(i) for i in range(num_classes)]
    plt.figure(figsize=(6, 5))
    for i, activity in enumerate(activity_names):
        mask = (labels_sampled == i)
        if np.any(mask):
            plt.scatter(features_2d[mask, 0], features_2d[mask, 1],
                       color=colors[i], marker='o', s=19, alpha=0.6, label=activity)
    plt.legend(title="Activities", fontsize=8, loc='best', ncol=2)
    plt.xlabel("t-SNE Component 1", fontsize=12)
    plt.ylabel("t-SNE Component 2", fontsize=12)
    plt.title("t-SNE Visualization", fontsize=14)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()


def plot_confusion_matrix(cm, activity_names, save_path):
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    num_classes = len(activity_names)
    labels = []
    for name in activity_names:
        if len(name) > 15:
            words = name.split()
            if len(words) > 1:
                mid = len(words) // 2
                labels.append(' '.join(words[:mid]) + '\n' + ' '.join(words[mid:]))
            else:
                labels.append(name[:15] + '\n' + name[15:])
        else:
            labels.append(name)
    df = pd.DataFrame(cm_normalized, index=labels, columns=labels)
    annot = df.copy().astype(str)
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            v = df.iloc[i, j]
            annot.iloc[i, j] = f"{v:.2f}"
    figsize = max(8, num_classes * 0.5)
    plt.figure(figsize=(figsize, figsize))
    sns.heatmap(df, annot=annot.values, fmt="", cmap="Blues", cbar=True,
                annot_kws={"size": max(6, 12 - num_classes // 3)}, vmin=0, vmax=1)
    plt.xticks(rotation=45, ha='right', fontsize=max(6, 11 - num_classes // 4))
    plt.yticks(rotation=0, fontsize=max(6, 11 - num_classes // 4))
    for spine in plt.gca().spines.values():
        spine.set_visible(True)
        spine.set_linewidth(0.5)
        spine.set_edgecolor('black')
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.title('Confusion Matrix', fontsize=14)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()


def evaluate(model, test_loader, device, use_amp=True):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
            if use_amp:
                with autocast('cuda'):
                    logits = model(batch_x)
            else:
                logits = model(batch_x)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    return acc, f1, precision, recall


def extract_features_and_predictions(model, data_loader, device, use_amp=True):
    model.eval()
    all_features, all_preds, all_labels = [], [], []
    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
            if use_amp:
                with autocast('cuda'):
                    features = model.extract_features(batch_x)
                    logits = model(batch_x)
            else:
                features = model.extract_features(batch_x)
                logits = model(batch_x)
            preds = torch.argmax(logits, dim=1)
            all_features.append(features.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    return np.vstack(all_features), np.array(all_preds), np.array(all_labels)


def compute_flops_params(model, input_shape, device):
    model.eval()
    dummy_input = torch.randn(1, *input_shape).to(device)
    macs, params = profile(model, inputs=(dummy_input,), verbose=False)
    flops_m = macs * 2 / 1e6
    params_m = params / 1e6
    return flops_m, params_m


def measure_inference_time(model, input_shape, device, n_runs=100, warmup=10):
    model.eval()
    dummy_input = torch.randn(1, *input_shape).to(device)
    with torch.no_grad():
        for _ in range(warmup):
            _ = model(dummy_input)
    if device.type == 'cuda':
        torch.cuda.synchronize()
    start = time.time()
    with torch.no_grad():
        for _ in range(n_runs):
            _ = model(dummy_input)
    if device.type == 'cuda':
        torch.cuda.synchronize()
    end = time.time()
    return (end - start) / n_runs * 1000


def train_single_dataset(dataset_name, X_train, y_train, X_test, y_test, activity_names, device,
                         ablation_config, epochs=50):
    seq_len = X_train.shape[1]
    input_dim = X_train.shape[2]
    num_classes = len(np.unique(y_train))
    sensor_config = SENSOR_CONFIG[dataset_name]
    train_dataset = UCIHARDataset(X_train, y_train)
    test_dataset = UCIHARDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True,
                             num_workers=4, pin_memory=True, prefetch_factor=2, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False,
                            num_workers=4, pin_memory=True, prefetch_factor=2, persistent_workers=True)
    model = MIELHAR(
        input_dim=input_dim,
        hidden_dim=128,
        num_classes=num_classes,
        sensor_config=sensor_config,
        use_physics=ablation_config['use_physics'],
        learnable_physics=ablation_config['learnable_physics'],
        use_physics_gradient=ablation_config['use_physics_gradient'],
        use_landscape=ablation_config['use_landscape'],
        use_energy_loss=ablation_config['use_energy_loss']
    ).to(device)
    flops_m, params_m = compute_flops_params(model, (seq_len, input_dim), device)
    inf_time = measure_inference_time(model, (seq_len, input_dim), device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    scaler = GradScaler('cuda')
    best_acc = 0.0
    best_metrics = {}
    best_model_state = None
    print(f"\n{'='*60}")
    print(f"Training {dataset_name} - {ablation_config['name']}")
    print(f"{'='*60}")
    print(f"Params: {params_m:.2f}M | FLOPs: {flops_m:.2f}M | Inf: {inf_time:.2f}ms")
    print(f"Input: ({seq_len}, {input_dim}) | Classes: {num_classes}")
    print(f"{'Epoch':<8} {'LR':<10} {'Loss':<10} {'Acc':<10} {'F1':<10} {'Prec':<10} {'Rec':<10}")
    print("-" * 60)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        annealing_weight = min(1.0, epoch / (epochs * 0.5))
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast('cuda'):
                logits, energy, gradient = model(batch_x, return_energy=True)
                cls_loss = criterion(logits, batch_y)
                if ablation_config['use_energy_loss']:
                    energy_loss = model.compute_energy_loss(energy, gradient, batch_y)
                    loss = cls_loss + model.energy_reg_weight * annealing_weight * energy_loss
                else:
                    loss = cls_loss
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        scheduler.step()
        test_acc, test_f1, test_prec, test_rec = evaluate(model, test_loader, device)
        if test_acc > best_acc:
            best_acc = test_acc
            best_metrics = {
                'acc': test_acc,
                'f1': test_f1,
                'prec': test_prec,
                'rec': test_rec
            }
            best_model_state = model.state_dict().copy()
            best_msg = " (*)"
        else:
            best_msg = ""
        current_lr = optimizer.param_groups[0]['lr']
        if epoch % 5 == 0 or epoch == epochs - 1:
            print(f"{epoch+1:<8} {current_lr:<10.6f} {total_loss/len(train_loader):<10.4f} "
                  f"{test_acc:<10.4f} {test_f1:<10.4f} {test_prec:<10.4f} {test_rec:<10.4f}{best_msg}")
        if device.type == 'cuda' and epoch % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()
    print("-" * 60)
    print(f"Best - Acc: {best_metrics['acc']:.4f} | F1: {best_metrics['f1']:.4f} | Prec: {best_metrics['prec']:.4f} | Rec: {best_metrics['rec']:.4f}")
    print(f"Params: {params_m:.2f}M | FLOPs: {flops_m:.2f}M | Inf: {inf_time:.2f}ms")
    model.load_state_dict(best_model_state)
    features, preds, labels = extract_features_and_predictions(model, test_loader, device)
   # cm = confusion_matrix(labels, preds)
    #tsne_path = f"{dataset_name}_{ablation_config['name']}_tsne.png"
    #cm_path = f"{dataset_name}_{ablation_config['name']}_confusion_matrix.png"
    print(f"\nGenerating t-SNE plot for {dataset_name} - {ablation_config['name']}...")
    #plot_tsne(features, labels, activity_names, tsne_path)
    #print(f"Generating Confusion Matrix for {dataset_name} - {ablation_config['name']}...")
    #plot_confusion_matrix(cm, activity_names, cm_path)
    if device.type == 'cuda':
        torch.cuda.empty_cache()
        gc.collect()
    return {
        'Dataset': dataset_name,
        'Config': ablation_config['name'],
        'Params(M)': round(params_m, 2),
        'FLOPs(M)': round(flops_m, 2),
        'Inf(ms)': round(inf_time, 2),
        'Acc': round(best_metrics['acc'], 4),
        'F1': round(best_metrics['f1'], 4),
        'Prec': round(best_metrics['prec'], 4),
        'Rec': round(best_metrics['rec'], 4)
    }


if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("=" * 80)
    print("MIEL-HAR Multi-Dataset Ablation Study (GPU Optimized)")
    print("=" * 80)
    print(f"Device: {device}\n")
    if device.type == 'cuda':
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"cuDNN Enabled: {torch.backends.cudnn.enabled}")
        print(f"cuDNN Benchmark: {torch.backends.cudnn.benchmark}\n")

    datasets_config = [
        ('UCI-HAR', load_uci_har, '/content/drive/MyDrive/HAR_Dataset/UCI'),
        ('WISDM', load_wisdm_data, '/content/drive/MyDrive/HAR_Dataset/WISDM'),
        ('PAMAP2', load_pamap2_data, '/content/drive/MyDrive/HAR_Dataset/PAMAP2'),
        ('MHEALTH', load_mhealth_data, '/content/drive/MyDrive/HAR_Dataset/MHEALTH'),
        ('MOBIACT', load_mobiact_data, '/content/drive/MyDrive/HAR_Dataset/MOBIACT'),
        ('MOTIONSENSE', load_motionsense_data, '/content/drive/MyDrive/HAR_Dataset/MOTIONSENSE'),
        ('UNIMIB', load_unimib_shar_data, '/content/drive/MyDrive/HAR_Dataset/UNIMIB (1)')
    ]

    ablation_configs = [
        {
            'name': 'Full Model (Ours)',
            'use_physics': True,
            'learnable_physics': True,
            'use_physics_gradient': True,
            'use_landscape': True,
            'use_energy_loss': True
        },
        {
            'name': 'w/o Physics Priors',
            'use_physics': False,
            'learnable_physics': False,
            'use_physics_gradient': False,
            'use_landscape': True,
            'use_energy_loss': True
        },
        {
            'name': 'Fixed Physics',
            'use_physics': True,
            'learnable_physics': False,
            'use_physics_gradient': True,
            'use_landscape': True,
            'use_energy_loss': True
        },
        {
            'name': 'w/o Physics Gradient',
            'use_physics': True,
            'learnable_physics': True,
            'use_physics_gradient': False,
            'use_landscape': True,
            'use_energy_loss': True
        },
        {
            'name': 'w/o Landscape Enc',
            'use_physics': True,
            'learnable_physics': True,
            'use_physics_gradient': True,
            'use_landscape': False,
            'use_energy_loss': True
        },
        {
            'name': 'w/o Energy Loss',
            'use_physics': True,
            'learnable_physics': True,
            'use_physics_gradient': True,
            'use_landscape': True,
            'use_energy_loss': False
        }
    ]

    all_datasets = {}
    print("=" * 80)
    print("Loading All Datasets")
    print("=" * 80)
    for dataset_name, loader_func, data_path in datasets_config:
        try:
            print(f"\n{'#'*60}")
            print(f"Loading {dataset_name}...")
            print(f"{'#'*60}")
            data = loader_func(data_path)
            if data is None or data[0] is None:
                print(f"Failed to load {dataset_name}")
                continue
            X_train, y_train, X_test, y_test, activity_names = data
            num_train_classes = len(np.unique(y_train))
            num_test_classes = len(np.unique(y_test))
            print(f"Train: {X_train.shape} | Classes: {num_train_classes}")
            print(f"Test: {X_test.shape} | Classes: {num_test_classes}")
            print(f"Activity Names: {activity_names}")
            train_class_dist = Counter(y_train)
            test_class_dist = Counter(y_test)
            print(f"Train class distribution: {dict(sorted(train_class_dist.items()))}")
            print(f"Test class distribution: {dict(sorted(test_class_dist.items()))}")
            all_datasets[dataset_name] = {
                'X_train': X_train,
                'y_train': y_train,
                'X_test': X_test,
                'y_test': y_test,
                'activity_names': activity_names
            }
            gc.collect()
        except Exception as e:
            print(f"Error loading {dataset_name}: {e}")
            import traceback
            traceback.print_exc()
            continue
    print(f"\n{'='*80}")
    print(f"Successfully loaded {len(all_datasets)} datasets")
    print(f"{'='*80}\n")

    results = []
    for dataset_name, dataset_data in all_datasets.items():
        for ablation_config in ablation_configs:
            try:
                result = train_single_dataset(
                    dataset_name,
                    dataset_data['X_train'],
                    dataset_data['y_train'],
                    dataset_data['X_test'],
                    dataset_data['y_test'],
                    dataset_data['activity_names'],
                    device,
                    ablation_config,
                    epochs=50
                )
                results.append(result)
                if device.type == 'cuda':
                    torch.cuda.empty_cache()
                    gc.collect()
            except Exception as e:
                print(f"Error training {dataset_name} with {ablation_config['name']}: {e}")
                import traceback
                traceback.print_exc()
                continue
    if results:
        print(f"\n{'='*80}")
        print("FINAL RESULTS - ABLATION STUDY")
        print(f"{'='*80}\n")
        results_df = pd.DataFrame(results)
        results_df = results_df[['Dataset', 'Config', 'Params(M)', 'FLOPs(M)', 'Inf(ms)', 'Acc', 'F1', 'Prec', 'Rec']]
        print(results_df.to_string(index=False))
        print(f"\n{'='*80}")
        for dataset_name in all_datasets.keys():
            dataset_results = results_df[results_df['Dataset'] == dataset_name]
            if len(dataset_results) > 0:
                print(f"\n{dataset_name} Results:")
                print(dataset_results[['Dataset', 'Config', 'Params(M)', 'FLOPs(M)', 'Inf(ms)', 'Acc', 'F1', 'Prec', 'Rec']].to_string(index=False))
        print(f"\n{'='*80}")
        try:
            display(results_df)
        except:
            pass

MIEL-HAR Multi-Dataset Ablation Study (GPU Optimized)
Device: cuda

GPU: Tesla T4
CUDA Version: 12.6
cuDNN Enabled: True
cuDNN Benchmark: True

Loading All Datasets

############################################################
Loading UCI-HAR...
############################################################

[UCI-HAR] Loading from: /content/drive/MyDrive/HAR_Dataset/UCI
  [1/9] Loading: total_acc_x_train.txt
  [2/9] Loading: total_acc_y_train.txt
  [3/9] Loading: total_acc_z_train.txt
  [4/9] Loading: body_acc_x_train.txt
  [5/9] Loading: body_acc_y_train.txt
  [6/9] Loading: body_acc_z_train.txt
  [7/9] Loading: body_gyro_x_train.txt
  [8/9] Loading: body_gyro_y_train.txt
  [9/9] Loading: body_gyro_z_train.txt
  [1/9] Loading: total_acc_x_test.txt
  [2/9] Loading: total_acc_y_test.txt
  [3/9] Loading: total_acc_z_test.txt
  [4/9] Loading: body_acc_x_test.txt
  [5/9] Loading: body_acc_y_test.txt
  [6/9] Loading: body_acc_z_test.txt
  [7/9] Loading: body_gyro_x_test.txt
  [8/9] Loading: b

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Training UCI-HAR - Full Model (Ours)
Params: 0.14M | FLOPs: 31.69M | Inf: 2.33ms
Input: (128, 9) | Classes: 6
Epoch    LR         Loss       Acc        F1         Prec       Rec       
------------------------------------------------------------
1        0.000999   1.0115     0.8806     0.8793     0.8838     0.8797     (*)
6        0.000965   0.5165     0.9172     0.9178     0.9179     0.9186    
11       0.000885   0.4949     0.9247     0.9243     0.9301     0.9246    
16       0.000768   0.4727     0.9260     0.9253     0.9287     0.9259    
21       0.000624   0.4579     0.9226     0.9214     0.9300     0.9219    
26       0.000469   0.4509     0.9298     0.9293     0.9303     0.9301    
31       0.000316   0.4470     0.9277     0.9272     0.9292     0.9278    
36       0.000181   0.4428     0.9274     0.9269     0.9278     0.9278    
41       0.000078   0.4367     0.9298     0.9291     0.9306     0.9299    
46       0.000016   0.4350     0.9304     0.9299     0.9312     0.9307    

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


1        0.000999   0.9600     0.8962     0.8955     0.9039     0.8938     (*)
6        0.000965   0.5145     0.9179     0.9180     0.9193     0.9185     (*)
11       0.000885   0.4968     0.9220     0.9217     0.9269     0.9220    
16       0.000768   0.4750     0.9379     0.9380     0.9384     0.9383     (*)
21       0.000624   0.4618     0.9301     0.9301     0.9303     0.9302    
26       0.000469   0.4506     0.9420     0.9417     0.9438     0.9417    
31       0.000316   0.4427     0.9505     0.9501     0.9519     0.9503     (*)
36       0.000181   0.4377     0.9416     0.9418     0.9434     0.9419    
41       0.000078   0.4347     0.9420     0.9422     0.9431     0.9425    
46       0.000016   0.4338     0.9420     0.9421     0.9432     0.9424    
50       0.000000   0.4332     0.9410     0.9411     0.9421     0.9414    
------------------------------------------------------------
Best - Acc: 0.9505 | F1: 0.9501 | Prec: 0.9519 | Rec: 0.9503
Params: 0.14M | FLOPs: 31.62M | Inf: 

...

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Training UNIMIB - w/o Physics Gradient
Params: 0.23M | FLOPs: 1.33M | Inf: 1.79ms
Input: (3, 151) | Classes: 17
Epoch    LR         Loss       Acc        F1         Prec       Rec       
------------------------------------------------------------
1        0.000999   2.2926     0.4690     0.2887     0.3678     0.3163     (*)
6        0.000965   1.1520     0.7732     0.6737     0.6890     0.6733     (*)
11       0.000885   0.9125     0.8161     0.7498     0.7696     0.7423    
16       0.000768   0.7799     0.8322     0.7796     0.7908     0.7746    
21       0.000624   0.7229     0.8364     0.7847     0.8056     0.7759    
26       0.000469   0.6833     0.8483     0.7982     0.8109     0.7917    
31       0.000316   0.6632     0.8471     0.7979     0.8096     0.7905    
36       0.000181   0.6507     0.8522     0.8087     0.8219     0.8002    
41       0.000078   0.6451     0.8530     0.8066     0.8190     0.7984    
46       0.000016   0.6422     0.8509     0.8057     0.8204     0.79

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Traceback (most recent call last):
  File "/tmp/ipython-input-3891870173.py", line 1177, in <cell line: 0>
    result = train_single_dataset(
             ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-3891870173.py", line 976, in train_single_dataset
    flops_m, params_m = compute_flops_params(model, (seq_len, input_dim), device)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-3891870173.py", line 929, in compute_flops_params
    macs, params = profile(model, inputs=(dummy_in

Error training UNIMIB with w/o Landscape Enc: Given normalized_shape=[128], expected input with shape [*, 128], but got input of size[1, 128, 3]

Training UNIMIB - w/o Energy Loss
Params: 0.23M | FLOPs: 1.33M | Inf: 2.23ms
Input: (3, 151) | Classes: 17
Epoch    LR         Loss       Acc        F1         Prec       Rec       
------------------------------------------------------------
1        0.000999   2.3434     0.4312     0.2164     0.3135     0.2674     (*)
6        0.000965   1.1474     0.7638     0.6604     0.6766     0.6575     (*)
11       0.000885   0.9022     0.8186     0.7528     0.7688     0.7460     (*)
16       0.000768   0.7706     0.8258     0.7629     0.7777     0.7550    
21       0.000624   0.7056     0.8322     0.7808     0.7923     0.7743    
26       0.000469   0.6739     0.8407     0.7942     0.8043     0.7895    
31       0.000316   0.6552     0.8445     0.7982     0.8098     0.7907     (*)
36       0.000181   0.6448     0.8407     0.7930     0.8036     0.7870

,Dataset,Config,Params(M),FLOPs(M),Inf(ms),Acc,F1,Prec,Rec
0,UCI-HAR,Full Model (Ours),0.14,31.69,2.33,0.9325,0.9320,0.9328,0.9327
1,UCI-HAR,w/o Physics Priors,0.14,31.62,1.43,0.9505,0.9501,0.9519,0.9503
2,UCI-HAR,Fixed Physics,0.14,31.69,2.34,0.9477,0.9479,0.9489,0.9478
3,UCI-HAR,w/o Physics Gradient,0.14,31.69,1.98,0.9481,0.9477,0.9493,0.9479
4,UCI-HAR,w/o Landscape Enc,0.15,34.31,2.02,0.9671,0.9677,0.9695,0.9675
5,UCI-HAR,w/o Energy Loss,0.14,31.69,2.30,0.9481,0.9486,0.9490,0.9492
6,WISDM,Full Model (Ours),0.14,19.20,2.05,0.9936,0.9904,0.9903,0.9905
7,WISDM,w/o Physics Priors,0.14,19.16,1.13,0.9933,0.9895,0.9893,0.9898
8,WISDM,Fixed Physics,0.14,19.20,2.87,0.9939,0.9904,0.9909,0.9899
9,WISDM,w/o Physics Gradient,0.14,19.20,1.76,0.9929,0.9894,0.9900,0.9888
